In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ashishpandey2062/next-word-predictor-text-generator-dataset")

print("Path to dataset files:", path)

100%|██████████| 61.5k/61.5k [00:00<00:00, 1.14MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/ashishpandey2062/next-word-predictor-text-generator-dataset/versions/1


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
txt_path = os.path.join(path,os.listdir(path)[0])

In [ ]:
with open(txt_path,'r',encoding='charmap') as file:
    text=file.read()

In [ ]:
text = text.lower()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

In [ ]:
input_sequences =[]
for sentence in text.split('\n'):
    tokenised_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1,len(tokenised_sentence)):
        input_sequences.append(tokenised_sentence[:i+1])

In [ ]:
len(input_sequences)

26383

In [ ]:
max_len = max([len(x) for x in input_sequences])

In [ ]:
padded_input_sequences = pad_sequences(input_sequences,max_len,padding='pre')

In [ ]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,    1,  155],
       [   0,    0,    0, ...,    1,  155,   21],
       [   0,    0,    0, ...,  155,   21, 2368],
       ...,
       [   0,    0,    0, ..., 2331,  290,   19],
       [   0,    0,    0, ...,  290,   19,   54],
       [   0,    0,    0, ...,   19,   54, 1535]], dtype=int32)

In [ ]:
X = padded_input_sequences[:,:-1]
y = padded_input_sequences[:,-1]

In [ ]:
y = to_categorical(y, num_classes=4994).astype(np.float32)

In [ ]:
y.shape

(26383, 4994)

In [ ]:
X.shape

(26383, 324)

In [ ]:
model = Sequential()
model.add(Embedding(4994,100,input_length = 324 ))
model.add(LSTM(150))
#model.add(LSTM(150))
model.add(Dense(4994, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.build(input_shape=(None, 324))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 324, 100)       │       499,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4994)           │       754,094 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,404,094 (5.36 MB)

 Trainable params: 1,404,094 (5.36 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X,y,epochs=3)

Epoch 1/3
825/825 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - accuracy: 0.0506 - loss: 7.2960
Epoch 2/3
825/825 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.0756 - loss: 6.3860
Epoch 3/3
825/825 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.0993 - loss: 5.9213


In [ ]:
loss, accuracy = model.evaluate(X, y)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

825/825 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.1332 - loss: 5.5272
Test Accuracy: 11.56%


In [ ]:
import time
text = "I don't care! Game's over!"

for i in range(15):
  token_text = tokenizer.texts_to_sequences([text])[0]
  padded_token_text = pad_sequences([token_text], maxlen= 324, padding='pre')
  pos = np.argmax(model.predict(padded_token_text))
  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
I don't care! Game's over! the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
I don't care! Game's over! the phone
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
I don't care! Game's over! the phone i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
I don't care! Game's over! the phone i know
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
I don't care! Game's over! the phone i know i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
I don't care! Game's over! the phone i know i know
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
I don't care! Game's over! the phone i know i know i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
I don't care! Game's over! the phone i know i know i know
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
I don't care! Game's over! the phone i know i know i know i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
I don't care! Game's over! the phone i know i know i know i know
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
I don't care! Game's over! the phone i know i know i know i know i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45m